# Day 2 - Building AI UIs with Gradio: Quick Prototyping for LLM Engineers

### Summary

This lecture introduces Gradio, a tool for creating user interfaces for machine learning models, specifically focusing on its application with large language models (LLMs) like GPT, Claude, and Gemini. The session aims to demonstrate how to quickly build and deploy interactive prototypes for business applications.

### Highlights

- 🚀 Gradio, part of Hugging Face, simplifies UI creation for machine learning models.
- 💡 It allows data scientists and LLM engineers to rapidly prototype and share applications.
- 🛠️ The lecture will cover building UIs for API calls to various LLMs and for a brochure application.
- 📈 Streaming and markdown will be integrated into the UI for enhanced functionality.
- 🤝 Gradio is a tool acquired by Hugging Face to facilitate the development of machine learning apps.

### Code Examples

```python
import gradio as gr

def greet(name):
return "Hello " + name

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch() 1
```

# Day 2 - Gradio Tutorial: Create Interactive AI Interfaces for OpenAI GPT Models

### **Summary**

This lecture introduces Gradio, a simple framework for building user interfaces. It demonstrates how to create basic UIs, integrate them with functions (including LLM calls), and share them via public URLs. The session highlights the ease of use and powerful capabilities of Gradio for rapid prototyping and collaboration.

### **Highlights**

- 🎉 Introduction to Gradio for building user interfaces easily.
- 💻 Demonstration of creating a simple UI to uppercase text using the `shout` function.
- 🌐 Explanation of how Gradio runs a local web server, enabling both local and public sharing of UIs.
- 🔗 Showing how to share UIs via public URLs using `share=True`.
- 🛠️ Example of creating a more configurable UI with labeled text boxes for input and output.
- 🤖 Integration of Gradio with an LLM (GPT-4 mini) to build a UI that interacts with the model.
- 🚀 Emphasizing the simplicity and power of Gradio for rapid prototyping and collaboration.

### **Code Examples**

```python
  import gradio as gr

  def shout(text):
      return text.upper()

  gr.Interface(fn=shout, inputs="text", outputs="text", allow_flagging="never").launch(share=True)

```

```python
  import gradio as gr
  import os
  import openai
  from dotenv import load_dotenv

  load_dotenv()
  openai.api_key = os.getenv("OPENAI_API_KEY")

  system_message = "You are a helpful assistant."

  def message_gpt(prompt):
      messages = [
          {"role": "system", "content": system_message},
          {"role": "user", "content": prompt}
      ]
      completion = openai.ChatCompletion.create(model="gpt-4", messages=messages)
      return completion.choices[0].message.content

  gr.Interface(fn=message_gpt, inputs=gr.Textbox(lines=6, label="Your Message"), outputs=gr.Textbox(lines=8, label="Response")).launch(share=True)

```

# Day 2 - Implementing Streaming Responses with GPT and Claude in Gradio UI

### **Summary**

This lecture builds upon the previous one, demonstrating how to enhance Gradio interfaces with markdown formatting and streaming capabilities for LLM responses. It showcases the ease of integrating these features and provides examples using both GPT-4 and Claude models.

### **Highlights**

- 📝 Integration of markdown formatting in Gradio outputs for cleaner, structured LLM responses.
- ⏩ Implementation of streaming LLM responses in Gradio using generator functions.
- 🔄 Explanation of cumulative result streaming to prevent output flickering in Gradio.
- 🤖 Demonstration of streaming with both GPT-4 and Claude models, highlighting API differences.
- 💡 Emphasizing Gradio's automatic handling of generators for streaming output.
- 🎨 Showcasing the creation of user-friendly interfaces with formatted and real-time responses.
- 🚀 Preparing to add model selection capabilities in the next session.

### **Code Examples**

```python
  import gradio as gr
  import os
  import openai
  from dotenv import load_dotenv

  load_dotenv()
  openai.api_key = os.getenv("OPENAI_API_KEY")

  system_message = "You are a helpful assistant."

  def stream_gpt(prompt):
      messages = [
          {"role": "system", "content": system_message},
          {"role": "user", "content": prompt}
      ]
      completion = openai.ChatCompletion.create(model="gpt-4", messages=messages, stream=True)
      result = ""
      for chunk in completion:
          if chunk.choices[0].delta.content:
              result += chunk.choices[0].delta.content
              yield result

  gr.Interface(fn=stream_gpt, inputs="text", outputs="markdown").launch(share=True)

```

```python
  import gradio as gr
  import anthropic
  import os
  from dotenv import load_dotenv

  load_dotenv()
  anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

  client = anthropic.Anthropic(api_key=anthropic_api_key)
  system_message = "You are a helpful assistant."

  def stream_claude(prompt):
      messages = [
          {"role": "user", "content": prompt}
      ]
      full_prompt = f"{anthropic.HUMAN_PROMPT} {prompt} {anthropic.AI_PROMPT}"
      with client.completions.stream(
          prompt=full_prompt,
          stop_sequences=[anthropic.HUMAN_PROMPT],
          model="claude-2",
          max_tokens_to_sample=1000,
          system=system_message,
      ) as stream:
          result = ""
          for text in stream.text:
              result += text
              yield result

  gr.Interface(fn=stream_claude, inputs="text", outputs="markdown").launch(share=True)

```

# Day 2 - Building a Multi-Model AI Chat Interface with Gradio: GPT vs Claude

### Summary

This lecture demonstrates how to create a versatile user interface using Gradio that allows users to select between different language models (GPT-4 and Claude) for generating company brochures from website landing pages. It also provides ideas for extending the functionality of the application.

### Highlights

- 🔄 Creating a hybrid generator function (`stream_model`) to switch between GPT-4 and Claude models.
- 💻 Building a Gradio interface with a dropdown to select between GPT-4 and Claude for generating responses.
- 📄 Integrating the company brochure generator from a previous lab into a Gradio interface.
- 🌐 Using `requests` and `BeautifulSoup` to scrape website content for brochure generation.
- 🤖 Generating company brochures using both GPT-4 and Claude models, showcasing different output styles.
- 💡 Suggesting enhancements like adding Gemini integration and tone selection for brochure generation.
- 🚀 Emphasizing the simplicity and effectiveness of Gradio for building complex applications.

### Code Examples

```python
import gradio as gr
  import os
  import openai
  import anthropic
  from dotenv import load_dotenv
  import requests
  from bs4 import BeautifulSoup

  load_dotenv()
  openai.api_key = os.getenv("OPENAI_API_KEY")
  anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
  client = anthropic.Anthropic(api_key=anthropic_api_key)

  class Website:
      def __init__(self, url):
          self.url = url
          self.title, self.text = self.get_context()

      def get_context(self):
          response = requests.get(self.url)
          soup = BeautifulSoup(response.text, 'html.parser')
          title = soup.title.string if soup.title else "No Title Found"
          for script in soup(["script", "style"]):
              script.decompose()
          text = soup.get_text()
          lines = (line.strip() for line in text.splitlines())
          chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
          text = '\n'.join(chunk for chunk in chunks if chunk)
          return title, text

  system_prompt = "You are an assistant that analyzes the contents of a company website landing page and creates a short brochure. Respond in markdown."

  def stream_gpt(prompt):
      messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": prompt}]
      completion = openai.ChatCompletion.create(model="gpt-4", messages=messages, stream=True)
      result = ""
      for chunk in completion:
          if chunk.choices[0].delta.content:
              result += chunk.choices[0].delta.content
              yield result

  def stream_claude(prompt):
      full_prompt = f"{anthropic.HUMAN_PROMPT} {prompt} {anthropic.AI_PROMPT}"
      with client.completions.stream(prompt=full_prompt, stop_sequences=[anthropic.HUMAN_PROMPT], model="claude-2", max_tokens_to_sample=1000, system=system_prompt) as stream:
          result = ""
          for text in stream.text:
              result += text
              yield result

  def stream_model(prompt, model):
      if model == "GPT":
          return stream_gpt(prompt)
      elif model == "Claude":
          return stream_claude(prompt)
      else:
          raise ValueError("Invalid model selected.")

  def stream_brochure(company_name, url, model):
      website = Website(url)
      prompt = f"Please generate a company brochure for {company_name}. Here is their landing page: {website.text}"
      return stream_model(prompt, model)

  gr.Interface(fn=stream_brochure, inputs=["text", "text", gr.Dropdown(["GPT", "Claude"], label="Select Model")], outputs="markdown").launch(share=True)
```

# Day 2 - Building Advanced AI UIs: From OpenAI API to Chat Interfaces with Gradio

### **Summary**

This lecture concludes the section on building user interfaces with Gradio, highlighting the skills acquired in integrating LLMs and creating interactive applications. It sets the stage for the next phase, which will focus on building chat UIs, enhancing prompt context, and developing a customer support assistant.

### **Highlights**

- 🎉 Congratulations on acquiring skills in using LLM APIs and building UIs with Gradio.
- 💻 Review of the functionalities developed, including model selection and dynamic content generation.
- 🚀 Preview of the next lecture, which will cover building chat UIs and enhancing prompt context.
- 🤖 Introduction to the upcoming project: developing a customer support assistant.
- 💡 Preparation for exploring more complex UI interactions and practical applications of LLMs.
- 🌟 Encouragement to continue learning and applying the newly acquired skills.